# உள்ளடக்க அட்டவணை
- [வழிகாட்டுதல் பற்றி](../../../../code/01.Introduce)
- [அமைப்பு](../../../../code/01.Introduce)
- [வரையறையில்லா உருவாக்கம்](../../../../code/01.Introduce)
- [Phi 3 பற்றி பேசுதல்](../../../../code/01.Introduce)
- [Regex](../../../../code/01.Introduce)
- [தேர்வு](../../../../code/01.Introduce)
- [சிந்தனை சங்கிலி](../../../../code/01.Introduce)
- [JSON உருவாக்கம்](../../../../code/01.Introduce)
- [HTML உருவாக்கம்](../../../../code/01.Introduce)


# Guidance பற்றி
Guidance என்பது எந்த மொழி மாதிரியின் (LM) வெளியீடுகளை கட்டுப்படுத்துவதற்கு நிரூபிக்கப்பட்ட திறந்த மூல Python நூலகமாகும். ஒரு API அழைப்பினால், நீங்கள் (Python இல்) மாதிரி பின்பற்ற வேண்டிய துல்லியமான நிரல்முறை கட்டுப்பாடுகளை குறிப்பிடலாம் மற்றும் JSON, Python, HTML, SQL, அல்லது பயன்பாட்டு தேவைக்கேற்ப எந்தவொரு கட்டமைப்பிலும் அமைந்த வெளியீட்டை உருவாக்க முடியும்.

Guidance பாரம்பரிய prompting நுட்பங்களிலிருந்து வேறுபடுகிறது.  இது inference அடுக்கில் மாதிரியை டோக்கன் தோறும் வழிநடத்தியால் கட்டுப்பாடுகளை அமல்படுத்துகிறது, இதனால் வெளியீடுகள் உயர் தரமானவை உருவாகி மிகவும் கட்டமைக்கப்பட்ட சூழல்களில் பயன்படுத்தும்போது செலவையும் தாமதத்தையும் 30–50% வரை குறைக்கிறது.

Guidance பற்றி மேலும் அறிய, GitHub இல் உள்ள [public repository on GitHub](https://github.com/guidance-ai/guidance) ஐப் பார்வையிடவும் அல்லது Microsoft Build இல் நடைபெற்ற [Guidance பிரேக்அவுட் அமர்வு](https://www.youtube.com/watch?v=qXMNPVVlCMs) ஐப் பார்க்கவும்.


# அமைப்பு
1. Guidance ஐ `pip install guidance --pre` கொண்டு நிறுவவும்
2. Azure இல் Phi 3.5 mini எண்ட்பாயிண்டை https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml என்ற தளத்திற்கு சென்று "Deploy" பொத்தானை கிளிக் செய்து பதிவேற்றவும்
3. உங்கள் எண்ட்பாயிண்டின் API திறவுகோலை `AZURE_PHI3_KEY` என்ற சுற்றுச்சூழல் மாறிலியில் மற்றும் URL ஐ `AZURE_PHI3_URL` என்ற சுற்றுச்சூழல் மாறிலியில் சேமிக்கவும்


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# கட்டுப்பாடில்லா உருவாக்கம்
எந்தவொரு கட்டுப்பாடும் இல்லாமல் உரையை `gen()` செயல்பாட்டைப் பயன்படுத்தி உருவாக்கலாம். இது Guidance இல்லாமல் மாடலைப் பயன்படுத்துவதே ஆகும்.

## உரையாடல் வடிவமைப்பு
பல்வேறு உரையாடல் மாடல்களைப் போலவே, Phi-3 ஒரு பயனர் மற்றும் உதவியாளர் இடையிலான செய்திகள் ஒரு குறிப்பிட்ட வடிவில் இருக்க வேண்டும் என்று எதிர்பார்க்கிறது. Guidance Phi-3-ன் உரையாடல் வார்ப்புருவை ஆதரிக்கிறது மற்றும் உங்களுக்காக உரையாடல் வடிவமைப்பை நிர்வகிக்கும்.

உரையாடல் முறைமைகளை உருவாக்க, உரையாடலின் ஒவ்வொரு பகுதியையும் `with user()` அல்லது `with assistant()` தொகுதியில் வைக்கவும். சிஸ்டம் செய்தியை அமைக்க `with system()` தொகுதியை பயன்படுத்தலாம்.


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## டோக்கன் சேமிப்புகள்
உயரான கட்டமைப்புள்ள சூழ்நிலைகளில், Guidance டோக்கன்களை தவிர்த்துவிட்டு தேவையான டோக்கன்களை மட்டும் உருவாக்க முடிகிறது, இதனால் செயல்திறன் மேம்படுகிறது, திறன் அதிகரிக்கிறது மற்றும் API செலவுகள் குறையும். உருவாக்கப்பட்ட டோக்கன்கள் இந்த நோட்புக்கில் ஹைலைட் செய்யப்பட்ட பின்னணியுடன் காட்டப்படுகின்றன. கட்டாயமான டோக்கன்கள் ஹைலைட் செய்யப்படாமல் காண்பிக்கப்படுகின்றன மற்றும் அவை உள்ளீட்டு டோக்கன்கள் போன்றவே செலவாகும்; உள்ளீட்டு டோக்கன்கள் வெளியீட்டு டோக்கன்களின் செலவின் ஒரு மூன்றாக மதிப்பிடப்படுகின்றன.

*குறிப்பு:* கட்டுப்பாடில்லாத உருவாக்கம் கொண்ட முதல் எடுத்துக்காட்டு எந்த டோக்கன்களையும் கட்டாயப்படுத்த முடியவில்லை, ஏனெனில் நாங்கள் எந்த விதமான கட்டுப்பாடுகளையும் வழங்கவில்லை.


# Phi 3 க்காக பேசுதல் 
With Guidance, நீங்கள் மாதிரியின் பதில்களில் உரையை எளிதாக செருகலாம். இது மாதிரியின் வெளியீட்டை ஒரு குறிப்பிட்ட திசையில் வழிநடத்த விரும்பினால் 도움이 될லாம்.


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

`ஆஸ்திரேலியாவின் தலைநகரம் ஆகும்` ஹைலைட் செய்யப்படவில்லை, ஏனெனில் உதவியாளரின் பதிலின் அந்த பகுதி வழிகாட்டலால் கட்டாயப்படுத்தப்பட்டது.


# regex-ஐப் பயன்படுத்தி கட்டுப்படுத்துதல்
முந்தைய உதாரணத்தில், Phi 3 `Canberra` என்று கேள்விக்கு பதில் கூறியபின் தொடர்ச்சியான விளக்கங்களுடன் பதிலளித்தது. மாடலின் வெளியீட்டை துல்லியமாக ஒரே ஒரு சொல் மட்டுமே ஆகக் கட்டுப்படுத்த ஒரு regex பயன்படுத்தலாம்.


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

regex-ஐ பயன்படுத்தும்போது, `Canberra` என்ற சொல் மட்டுமே உருவாக்கப்படுகிறது.


# பல தேர்வுகளில் இருந்து தேர்வு செய்வது
சில சாத்தியமான தேர்வுகள் தெரியுமானால், மாதிரியை விருப்பங்களின் பட்டியலிலிருந்து தேர்வு செய்ய `select()` செயல்பாட்டைப் பயன்படுத்தலாம்.


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

`select()` உடன், `Can` என்ற டோக்கன் மட்டுமே உருவாக்கப்பட்டது. `Canberra` என்பது பதிலை முடிக்கக்கூடிய ஒரே தேர்வே என்பதால், மீதமுள்ள டோக்கன்கள் கட்டாயமாக உருவாக்கப்பட்டன.


# சிந்தனைத் தொடர்

சிந்தனைத் தொடர் என்பது ஒரு பிரச்சினையை படி படியாக செயலாற்ற ஊக்குவிப்பதன் மூலம் மாதிரியின் வெளியீட்டு தரத்தை மேம்படுத்த உதவக்கூடிய நுட்பம் ஆகும். சாதாரணமாக, இறுதி பதிலைப் பெற பல முறை முன்மொழிவு சுற்றுகள் (prompt turns) தேவையாக இருக்கும். முதலில், மாதிரியை படி படியாக சிந்திக்க சொல்லுங்கள். பிறகு, இறுதி பதிலை வழங்க மாதிரியை மீண்டும் கேளுங்கள். சாதாரண chat inference API-களில், இதற்கு 2 API அழைப்புகள் தேவைப்படும், மற்றும் மாதிரி உருவாக்கிய “சிந்தனைத் தொடர்” இருமுறை கட்டணம் விதிக்கப்படுகிறது — ஒரு முறை மாதிரி அதை உருவாக்கும்போது வெளியீட்டு டோக்கன்களாக, பின்னர் இரண்டாம் அழைப்புக்காக உள்ளீட்டு டோக்கன்களாக. Guidance,  மூலம், முழு பல-படி செயல்முறை ஒரே API அழைப்பின் பகுதியாக செயலாக்கப்பட்டு கட்டணம் வசூலிக்கப்படுகிறது, இதனால் செலவும் தாமதமும் குறையும்.


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# JSON உருவாக்கம்
வழிகாட்டுதலால் JSON ஸ்கீமா அல்லது pydantic மாதிரிக்கு இணக்கமான JSON உருவாக்கப்படுவதை உறுதிசெய்யலாம், உதாரணமாக இங்கு காண்பிக்கப்பட்டுள்ள பயனர் சுயவிவர ஸ்கீமா.


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## HTML உருவாக்கம்

Guidance-ஐ குறியீடு உருவாக்கவும் மற்றும் நிரலாக்க மொழியின் இலக்கண (syntax) தேவைகளை பின்பற்றவும் பயன்படுத்தலாம். இந்த பகுதியில், மிகவும் எளிய HTML வலைப்பக்கங்களை எழுத ஒரு சிறு Guidance திட்டத்தை நாங்கள் உருவாக்கப்போகிறோம்.

நாங்கள் வலைப்பக்கத்தை சிறிய பகுதிகளாகப் பிரித்து, ஒவ்வொரு பகுதிக்கும் தனித்துவமான Guidance செயல்பாட்டை உருவாக்குவோம்.  பின்னர் இவை எங்கள் இறுதி செயல்பாட்டில் ஒன்றிணைக்கப்பட்டு ஒரு HTML வலைப்பக்கத்தை உருவாக்குவோம்.
அதன்பிறகு இந்த செயல்பாட்டை Azure AI-வில் Guidance ஆதரவு கொண்ட ஒரு மாதிரியின் மீது இயக்குவோம்.

*குறிப்பு:* இது முழுமையாக அம்சசமயமான HTML உருவாக்கி அல்ல; நோக்கம் உங்கள் தனிப்பட்ட தேவைகளுக்கு ஏற்ப கட்டமைக்கப்பட்ட வெளியீட்டை நீங்கள் எப்படி உருவாக்கலாம் என்பதை காட்டுவதே.

நாங்கள் Guidance-இலிருந்து தேவைப்படும்வற்றை இறக்குமதி செய்வதிலிருந்து ஆரம்பிக்கிறோம்:


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

HTML வலைப்பக்கங்கள் மிகவும் கட்டமைக்கப்பட்டவை, மற்றும் நாங்கள் Guidance-ஐப் பயன்படுத்தி பக்கத்தின் அந்த பகுதிகளைக் 'force' செய்வோம்.
When we explicitly require text from the model, we need to ensure it doesn't include anything which could be a tag - that is, we must exclude the '<' and '>' characters:


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

பின்னர் இந்த செயல்பாட்டைப் பயன்படுத்தி எந்தவொரு HTML குறிச்சொல்லுக்குள் இருக்கும் உரையை உருவாக்கலாம்:


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

இப்போது, நாம் பக்க தலைப்பை உருவாக்குவோம்.
இதன் பகுதியாக, நமக்கு ஒரு பக்கத் தலைப்பை உருவாக்க வேண்டும்:


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

HTML பக்கத்தின் உட்பகுதி தலைப்புகள் மற்றும் பத்திகளால் நிரப்பப்படும்.
ஒவ்வொன்றையும் செய்ய ஒரு செயல்பாட்டை வரையறுக்கலாம்:


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

இப்போது, HTML உடலின் பகுதியை வரையறுக்கும் செயல்பாடு.
இது `select()` ஐ `recurse=True` உடன் பயன்படுத்தி பல தலைப்புகள் மற்றும் பத்திகளை உருவாக்குகிறது:


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

அடுத்து, முழுமையான HTML பக்கத்தை உருவாக்கும் செயல்பாட்டிற்கு வருகிறோம்.
நாம் HTML தொடக்க குறிச்சொல்லை சேர்க்கிறோம், பின்னர் header-ஐ உருவாக்கி, அதன் பிறகு body-ஐ உருவாக்கி, பின்னர் HTML முடிவு குறிச்சொல்லை இணைக்கிறோம்:


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

நாங்கள் பயனர்-நட்பான ஒரு wrapper ஐ வழங்குகிறோம், இது நமக்கு அனுமதிக்கும்:
- உருவாக்கத்தின் வெப்பநிலையை அமைக்க
- Model object இல் இருந்து உருவாக்கப்பட்ட பக்கத்தை பிடிக்க


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

நாம் மாடலுக்கு ஒரு தூண்டுதலை வழங்கி, பின்னர் ஒரு உருவாக்கத்தை கோரலாம்:


In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

நாம் பின்னர் வெளியீட்டை ஒரு கோப்பிற்கு எழுதலாம்:


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

மற்றும் [முடிவைக் பார்க்கவும்](../../../../code/01.Introduce/sample_page.html).


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
மறுப்பு:
இந்த ஆவணம் AI மொழிபெயர்ப்பு சேவை Co-op Translator (https://github.com/Azure/co-op-translator) மூலம் மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்தைக் காக்க முயற்சி செய்தாலும், தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறுகள் உள்ளிருக்கலாம் என்பதை கவனிக்கவும். மூல ஆவணம் அதன் சொந்த மொழியில் அதிகாரப்பூர்வ ஆதாரமாகக் கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பை பரிந்துரைக்கிறோம். இந்த மொழிபெயர்ப்பின் பயன்பாட்டால் ஏற்படும் எந்த தவறான புரிதலுக்கும் அல்லது தவறான விளக்கத்திற்கும் நாங்கள் பொறுப்பல்ல.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
